# Import Statements

### I linked the google sheet to this notebook using this source: https://towardsdatascience.com/from-google-sheet-to-your-jupyter-notebook-ccdbf28fbf1b. Any updates to the google sheet will be reflected automatically in this notebook

In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import datetime

In [2]:
#Importing the module
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# Link to Google Sheets

Connect the Google Sheets API

In [3]:
#The scope is always look like this so we did not need to change anything
scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#Name of our Service Account Key
google_key_file = 'jupyter-spring2020ctc-connect-3bde2ba97cd1.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# Spring 2020 Data Cleaning

Open and save the "Spring 2020 CTC Recording" Google Sheets document, specifically the "Data-Analysis" sheet

In [4]:
#Opening the worksheet by using Worksheet ID
spring20_spreadsheet_key = '1OiGBYsa9eK26lAaznmiiczgSX17GHlOWoS0crNza-vo'
spring20_workbook = gc.open_by_key(spring20_spreadsheet_key)

#Selecting which sheet to pulling the data
spring20_sheet = spring20_workbook.worksheet('Data-Analysis')

#Pulling the data and transform it to the data frame
spring20_values = spring20_sheet.get_all_values()
spring20 = pd.DataFrame(spring20_values[1:], columns = spring20_values[0])

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001B5120FEBC8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

Open the dataframe

In [ ]:
spring20.head()

Fill blank cells with NA's

In [ ]:
spring20.replace(r'', np.NaN, inplace=True)
spring20.head()

Replace the header with the second row of the dataframe

In [ ]:
new_header = spring20.iloc[0] # grab the first row for the header
spring20 = spring20[1:] # take the data less the header row
spring20.columns = new_header # set the header row as the spring20 header
spring20.head()

Drop "Current Total" column- we will not be needing this

In [ ]:
spring20.columns
spring20.drop(columns=['Current Total'], inplace=True) # drop irrelevant column
spring20.head()

Delete scrap notes at the bottom of the sheet

In [ ]:
spring20 = spring20.iloc[0:44,] # keep only relevant rows

Fill NA's in "Fraternity" column using the forward fill method as appropriate

In [ ]:
spring20['Fraternity'].fillna(method='ffill', inplace=True) # forward fill NA values for Fraternity name 
spring20.head()

Delete rows with empty weekly bags data (duplicate rows of each house caused by formats of the google sheet)

In [ ]:
spring20 = spring20[~spring20['Fraternity'].duplicated(keep=False) |
                    spring20[spring20.columns[1:]].notnull().any(axis=1)]
spring20.head()

Tidy the dataframe by melting the week columns into a better format

In [ ]:
spring20 = pd.melt(spring20, id_vars=['Fraternity'], value_vars=spring20.columns[1:])
spring20.columns=['Fraternity', 'Week', 'Bags']
spring20.head()

Convert "Week" column into standard date-time string, and convert to date-time type

In [ ]:
spring20['Week'] = spring20['Week'].apply(lambda x: x[x.find('(')+1:-1] + '/20')
spring20['Week']= pd.to_datetime(spring20['Week'].astype(str), format='%m/%d/%y')
spring20.head()

Fill NaN with 0's and convert "Bags" column to float type.

In [ ]:
spring20.fillna(0, inplace=True)
spring20['Bags'] = pd.to_numeric(spring20['Bags'])
spring20.head()

Add "Spring 2020" label. Enjoy clean and tidy data!

In [ ]:
spring20['Semester'] = 'Spring 2020'
spring20.head()

# Fall 2019 Data Cleaning

Open and save the "Fall 2019 CTC Recording" Google Sheets document, specifically the "Data-Analysis" sheet

In [ ]:
#Opening the worksheet by using Worksheet ID
fall19_spreadsheet_key = '14BYq15BfBUclNLbO8IRSAQeG98qCGDG3n59Hkfh3huU'
fall19_workbook = gc.open_by_key(fall19_spreadsheet_key)

#Selecting which sheet to pulling the data
fall19_sheet = fall19_workbook.worksheet('Data-Analysis')

#Pulling the data and transform it to the data frame
fall19_values = fall19_sheet.get_all_values()
fall19 = pd.DataFrame(fall19_values[1:], columns = fall19_values[0])

Open the dataframe

In [ ]:
fall19.head()

Fill blank cells with NA's

In [ ]:
fall19.replace(r'', np.NaN, inplace=True)
fall19.head()

Replace the header with the second row of the dataframe

In [ ]:
new_header = fall19.iloc[0] # grab the first row for the header
fall19 = fall19[1:] # take the data less the header row
fall19.columns = new_header # set the header row as the spring20 header
fall19.head()

Drop "Current Total" column- we will not be needing this

In [ ]:
fall19.columns
fall19.drop(columns=['Current Total'], inplace=True) # drop irrelevant column
fall19.head()

Delete scrap notes at the bottom of the sheet

In [ ]:
fall19 = fall19.iloc[0:47,] # keep only relevant rows

Fill NA's in "Fraternity" column using the forward fill method as appropriate

In [ ]:
fall19['Fraternity'].fillna(method='ffill', inplace=True) # forward fill NA values for Fraternity name 
fall19.head()

Delete rows with empty weekly bags data (duplicate rows of each house caused by formats of the google sheet)

In [ ]:
fall19 = fall19[~fall19['Fraternity'].duplicated(keep=False) |
                    fall19[fall19.columns[1:]].notnull().any(axis=1)]
fall19.head()

Tidy the dataframe by melting the week columns into a better format

In [ ]:
fall19 = pd.melt(fall19, id_vars=['Fraternity'], value_vars=fall19.columns[1:])
fall19.columns=['Fraternity', 'Week', 'Bags']
fall19.head()

Convert "Week" column into standard date-time string, and convert to date-time type

In [ ]:
fall19['Week'] = fall19['Week'].apply(lambda x: x[x.find('(')+1:-1] + '/19')
fall19['Week']= pd.to_datetime(fall19['Week'].astype(str), format='%m/%d/%y')
fall19.head()

Fill NaN with 0's and convert "Bags" column to float type.

In [ ]:
fall19.fillna(0, inplace=True)
fall19['Bags'] = pd.to_numeric(fall19['Bags'])
fall19.head()

Add "Fall 2019" label. Enjoy clean and tidy data!

In [ ]:
fall19['Semester'] = 'Fall 2019'
fall19.head()

# Spring 2019 Data Cleaning

Open and save the "Spring 2019 CTC Recording" Google Sheets document, specifically the "Data-Analysis" sheet

In [ ]:
#Opening the worksheet by using Worksheet ID
spring19_spreadsheet_key = '1tEiEe2UfWBQr1W2BncKevsvF-SG6XbLl-H1uDYgWF9w'
spring19_workbook = gc.open_by_key(spring19_spreadsheet_key)

#Selecting which sheet to pulling the data
spring19_sheet = spring19_workbook.worksheet('Data-Analysis')

#Pulling the data and transform it to the data frame
spring19_values = spring19_sheet.get_all_values()
spring19 = pd.DataFrame(spring19_values[1:], columns = spring19_values[0])

Open the dataframe

In [ ]:
spring19.head()

Fill blank cells with NA's

In [ ]:
spring19.replace(r'', np.NaN, inplace=True)
spring19.head()

Delete first row

In [ ]:
spring19 = spring19.iloc[1:,] # keep only relevant rows
spring19.head()

Transpose the dataframe (inverse rows and columns)

In [ ]:
spring19 = spring19.transpose()
spring19.head()

Make the index it's own column, labeled "Week"

In [ ]:
spring19['Week'] = spring19.index
spring19.head()

Tidy the dataframe by melting the columns into a better format

In [ ]:
spring19 = pd.melt(spring19, id_vars=['Week'], value_vars=spring19.columns[:-1])
spring19.drop(columns=['variable'], inplace=True) # drop irrelevant column
spring19.columns=['Week', 'Fraternity']
spring19.head()

Extract number of bags and fraternity label into own columns, convert bags to numeric type

In [ ]:
spring19['Bags'] = spring19['Fraternity'].str.split('- ', expand=True)[1]
spring19['Fraternity'] = spring19['Fraternity'].str.split('- ', expand=True)[0]
spring19.head()

Fill NAN in "Bags" column with 0's and convert column to float type

In [ ]:
spring19['Bags'].fillna(0, inplace=True)
spring19['Bags'] = pd.to_numeric(spring19['Bags'])
spring19.head()

Replace inconsistent fraternity labels with standard label

In [ ]:
spring19['Fraternity'].replace(['Kappa Sig (2 large bags back porch ', 'k sig ', 'ksig ', 'kappa sig '], 'Kappa Sig', inplace=True)
spring19['Fraternity'].replace(['aepi ', 'aepi'], 'AEPi', inplace=True)
spring19['Fraternity'].replace(['alpha sig ', 'a sig ', 'Alpha sig (@ house and wertland '], 'Alpha Sig', inplace=True)
spring19['Fraternity'].replace(['sig nu ', 'Sig Nu '], 'Sigma Nu', inplace=True)
spring19['Fraternity'].replace('chi phi ', 'Chi Phi', inplace=True)
spring19['Fraternity'].replace(['ka ', 'ka  '], 'KA', inplace=True)
spring19['Fraternity'].replace(['ATO (8 small bags ) ', 'ato '], 'ATO', inplace=True)
spring19['Fraternity'].replace('dke ', 'DKE', inplace=True)
spring19['Fraternity'].replace(['surp ', 'serp '], 'SERP', inplace=True)
spring19['Fraternity'].replace(['phi psi '], 'Phi Psi', inplace=True)
spring19['Fraternity'].replace('sig pi ', 'Sigma Pi', inplace=True)
spring19['Fraternity'].replace('hall ', 'The Hall', inplace=True)
spring19['Fraternity'].replace(['figi ', 'fiji ',], 'Fiji', inplace=True)
spring19['Fraternity'].replace('elmo ', 'Elmo', inplace=True)
spring19.head()

Remove rows with NAN values in the "Fraternity" column

In [ ]:
spring19 = spring19[spring19['Fraternity'].notnull()]
spring19.head()

Convert "Week" column into standard date-time string, and convert to date-time type

In [ ]:
spring19['Week'] = spring19['Week'].str.split(" ", 1, expand=True)[1] + ', 19'
spring19['Week']= pd.to_datetime(spring19['Week'].astype(str), format='%b, %d, %y')
spring19.head()

Add "Spring 2019" label. Enjoy clean and tidy data!

In [ ]:
spring19['Semester'] = 'Spring 2019'
spring19.head()

# Spring 2018 Data Cleaning

Open and save the "Copy of Spring 2018 Pickup Schedule" Google Sheets document, specifically the "Data-Analysis" sheet

In [ ]:
#Opening the worksheet by using Worksheet ID
spring18_spreadsheet_key = '1QMK71zz__7E9JucmevrADed7H7etXxCP9X9c9lnSNI4'
spring18_workbook = gc.open_by_key(spring18_spreadsheet_key)

#Selecting which sheet to pulling the data
spring18_sheet = spring18_workbook.worksheet('Data-Analysis')

#Pulling the data and transform it to the data frame
spring18_values = spring18_sheet.get_all_values()
spring18 = pd.DataFrame(spring18_values[1:], columns = spring18_values[0])

Open the dataframe

In [ ]:
spring18.head()

Fill blank cells with NA's

In [ ]:
spring18.replace(r'', np.NaN, inplace=True)
spring18.head()

Transpose the dataframe (inverse rows and columns)

In [ ]:
spring18 = spring18.transpose()
spring18.head()

Make the index it's own column, labeled "Week"

In [ ]:
spring18['Week'] = spring18.index
spring18.head()

Tidy the dataframe by melting the columns into a better format

In [ ]:
spring18 = pd.melt(spring18, id_vars=['Week'], value_vars=spring18.columns[:-1])
spring18.drop(columns=['variable'], inplace=True) # drop irrelevant column
spring18.columns=['Week', 'Fraternity']
spring18.head()

Extract number of bags and fraternity label into own columns, convert bags to numeric type

In [ ]:
spring18['Bags'] = spring18['Fraternity'].str.split(': ', expand=True)[1].str.split(' bag', expand=True)[0]
spring18['Fraternity'] = spring18['Fraternity'].str.split(': ', expand=True)[0]
spring18.head()

Fill NAN in "Bags" column with 0's and convert column to float type

In [ ]:
spring18['Bags'].fillna(0, inplace=True)
spring18['Bags'] = pd.to_numeric(spring18['Bags'])
spring18.head()

Replace inconsistent fraternity labels with standard label

In [ ]:
spring18['Fraternity'].replace('Pi Kapp', 'Pi Kappa Phi', inplace=True)
spring18['Fraternity'].replace('Alpha sig', 'Alpha Sig', inplace=True)
spring18['Fraternity'].replace('Sigma pi', 'Sigma Pi', inplace=True)
spring18.head()

Remove rows with NAN values in the "Fraternity" column

In [ ]:
spring18 = spring18[spring18['Fraternity'].notnull()]
spring18.head()

Convert "Week" column into standard date-time string, and convert to date-time type

In [ ]:
spring18['Week'] = spring18['Week'].str.split(" ", 1, expand=True)[1] + ' 18'
spring18['Week']= pd.to_datetime(spring18['Week'].astype(str), format='%b %d %y')
spring18.head()

Add "Spring 2018" label. Enjoy clean and tidy data!

In [ ]:
spring18['Semester'] = 'Spring 2018'
spring18.head()

# Fall 2017 Data Cleaning

Open and save the "Copy of  Pickup Schedule" Google Sheets document, specifically the "Data-Analysis" sheet

In [ ]:
#Opening the worksheet by using Worksheet ID
fall17_spreadsheet_key = '1Ew11cMZ7e1VhT0hYVpTf41dthT6JZzET-a2bD3yUiq4'
fall17_workbook = gc.open_by_key(fall17_spreadsheet_key)

#Selecting which sheet to pulling the data
fall17_sheet = fall17_workbook.worksheet('Data-Analysis')

#Pulling the data and transform it to the data frame
fall17_values = fall17_sheet.get_all_values()
fall17 = pd.DataFrame(fall17_values[1:], columns = fall17_values[0])

Open the dataframe

In [ ]:
fall17.head()

Fill blank cells with NA's

In [ ]:
fall17.replace(r'', np.NaN, inplace=True)
fall17.head()

Transpose the dataframe (inverse rows and columns)

In [ ]:
fall17 = fall17.transpose()
fall17.head()

Make the index it's own column, labeled "Week"

In [ ]:
fall17['Week'] = fall17.index
fall17.head()

Tidy the dataframe by melting the columns into a better format

In [ ]:
fall17 = pd.melt(fall17, id_vars=['Week'], value_vars=fall17.columns[:-1])
fall17.drop(columns=['variable'], inplace=True) # drop irrelevant column
fall17.columns=['Week', 'Fraternity']
fall17.head()

Extract number of bags and fraternity label into own columns, convert bags to numeric type

In [ ]:
fall17['Bags'] = fall17['Fraternity'].str.split(': ', expand=True)[1].str.split(' ', expand=True)[0]
fall17['Fraternity'] = fall17['Fraternity'].str.split(': ', expand=True)[0]
fall17.head()

Fill NAN in "Bags" column with 0's and convert column to float type

In [ ]:
fall17['Bags'].fillna(0, inplace=True)
fall17['Bags'] = pd.to_numeric(fall17['Bags'])
fall17.head()

Replace inconsistent fraternity labels with standard label

In [ ]:
fall17['Fraternity'].replace('Pi Kapp', 'Pi Kappa Phi', inplace=True)
fall17.head()

Remove rows with NAN values in the "Fraternity" column

In [ ]:
fall17 = fall17[fall17['Fraternity'].notnull()]
fall17.head()

Convert "Week" column into standard date-time string, and convert to date-time type

In [ ]:
fall17['Week'] = fall17['Week'].str.split(", ", 1, expand=True)[1] + ' 17'
fall17['Week']= pd.to_datetime(fall17['Week'].astype(str), format='%b %d %y')
fall17.head()

Add "Fall 2017" label. Enjoy clean and tidy data!

In [ ]:
fall17['Semester'] = 'Fall 2017'
fall17.head()

In [ ]:
fall17

### Quick Graphs for total bags across all houses

In [ ]:
totals = pd.concat([spring20, fall19, spring19, spring18, fall17], axis=0, sort=False)

In [ ]:
totals

In [ ]:
pd.set_option('display.max_rows', None)
totals

In [ ]:
totals['Bags'].sum()

In [ ]:
weekly = totals.groupby(['Week']).sum()

In [ ]:
weekly.reset_index(level=0, inplace=True)

In [ ]:
weekly['Cumulative Bags'] = weekly['Bags'].cumsum()

In [ ]:
sns.lineplot(weekly['Week'], weekly['Bags'], linewidth=2.0)

In [ ]:
sns.lineplot(weekly['Week'], weekly['Cumulative Bags'], linewidth=2.0)